# Knowledge Retention

Knowledge Retention ใช้สำหรับการประเมินเพื่อหาว่า LLM สามารถ เก็บข้อมูลตลอดการสนทนาได้หรือไม่

In [7]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings
import os

os.environ["OPENAI_API_VERSION"] = settings.OPENAI_API_VERSION
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = settings.AZURE_OPENAI_ENDPOINT

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
)
azure_openai = AzureOpenAI(model=custom_model)

/usr/local/lib/python3.9/site-packages/deepeval/__init__.py:42: UserWarning: You are using deepeval version 0.21.42, however version 0.21.45 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [8]:
from deepeval.test_case import LLMTestCase

messages = [
    LLMTestCase(
        input="",
        actual_output="Hello! I'm here to assist you with opening a new bank account. To start, may I have your full name, please?"
    ),
    LLMTestCase(
        input="Sure, it's Alex Johnson. I also have a dog called Jacky.",
        actual_output="Great, Alex! Now, could you please provide your current address?"
    ),
    LLMTestCase(
        input="123 Maple Street, Springfield.",
        actual_output="Is that all?"
    ),
    LLMTestCase(
        input="I also have another address at 123 Broadway, NYC.",
        actual_output="Wonderful. Next, I'll need your bank account number where we can link this new account."
    ),
    LLMTestCase(
        input="456789123",
        actual_output="This account number seems invalid, can you please double-check?"
    ),
    LLMTestCase(
        input="Sorry you're right, its 456789124.",
        actual_output="Thank you. And for our next step, could you tell me your date of birth?"
    ),
    LLMTestCase(
        input="It's July 9th.",
        actual_output="What about the year?"
    ),
    LLMTestCase(
        input="1990",
        actual_output="Got it. Now, for security purposes, could you share your mother's maiden name?"
    ),
    LLMTestCase(
        input="It's Smith.",
        actual_output="Excellent. Just a few more details. What is your phone number?"
    ),
    LLMTestCase(
        input="555-0102",
        actual_output="Great, we're almost done. Could you remind me of your full name for the account documentation?"
    ),
    LLMTestCase(
        input="Didn't I tell you already? It's Alex Johnson.",
        actual_output="What is your bank account number?"
    ),
    LLMTestCase(
        input="Yes, I did... It's 456789124. Are you not keeping track of this?",
        actual_output="One last question, what is the country code of your phone number?"
    ),
    LLMTestCase(
        input="+44",
        actual_output="Thank you, Alex, for bearing with me. We now have all the information we need to proceed with opening your new bank account. I appreciate your cooperation and patience throughout this process."
    )
]

In [10]:
from deepeval.metrics import KnowledgeRetentionMetric
from deepeval.test_case import ConversationalTestCase

test_case = ConversationalTestCase(messages=messages)
metric = KnowledgeRetentionMetric(threshold=0, model = azure_openai)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

Output()

0.6153846153846154
The knowledge retention score is 0.62 because the LLM response highlights forgetfulness by repeatedly asking for information that has already been provided earlier in the conversation, such as the bank account number, mother's maiden name, phone number, and full name. Despite the LLM suggesting that all necessary information has been provided, it is clear that key details are being overlooked or forgotten.


There are three optional parameters when creating a KnowledgeRetentionMetric:

- [Optional] threshold: a float representing the maximum passing threshold, defaulted to 0.5.
- [Optional] model: a string specifying which of OpenAI's GPT models to use, OR any custom LLM model of type DeepEvalBaseLLM. Defaulted to 'gpt-4o'.
- [Optional] include_reason: a boolean which when set to True, will include a reason for its evaluation score. Defaulted to True.
- [Optional] strict_mode: a boolean which when set to True, enforces a binary metric score: 1 for perfection, 0 otherwise. It also overrides the current threshold and sets it to 0. Defaulted to False.

### How Is It Calculated?

Knowledge Retention = Number of Messages without Knowledge Attritions/Total Number of Messages

KnowledgeRetentionMetric จะใช้ LLM ในการสกัด knowledge จาก message ก่อนที่จะใช้ LLM เดียวกันเพื่อพิจารณาว่าคำตอบของ LLM ที่สอดคล้องกับแต่ละรายการเพื่อหาว่าสามารถรู้จำบริบทเดิมได้หรือไม่

Note : metric นี้เป็นเพียงแค่ beta version